## Create table for device whitelist

When the provision of a new camera occurs, it will check this table in order to allow/deny provisioning for the device

In [ ]:
import boto3
from botocore.exceptions import ClientError

dynamodb = boto3.resource("dynamodb", region_name="eu-west-1")
TABLE_NAME = "device_whitelist"

try:
    table = dynamodb.create_table(
        TableName=TABLE_NAME,
        KeySchema=[
            {"AttributeName": "device_serial_number", "KeyType": "HASH"},
        ],
        AttributeDefinitions=[
            {"AttributeName": "device_serial_number", "AttributeType": "S"},
        ],
        BillingMode="PAY_PER_REQUEST"
    )
    print("Creating table...")
    table.wait_until_exists()
    print(f"Table '{TABLE_NAME}' created.")
except ClientError as e:
    if e.response["Error"]["Code"] == "ResourceInUseException":
        print(f"Table '{TABLE_NAME}' already exists.")
    else:
        raise

#### Add our current cameras in the whitelist

In [ ]:
from datetime import datetime, timezone

table = dynamodb.Table(TABLE_NAME)

serial_numbers = [
    "B8A44F976508",
    "B8A44F9C9902",
    "B8A44F9CA507",
    "B8A44FA0DC7A",
    "B8A44FB0AC74",
    "B8A44FB3A1F9",
    "B8A44FB97C3C",
    "B8A44FB981BB",
    "B8A44FCDF536",
    "B8A44FD014E5",
    "B8A44FE6C853",
    "B8A44FE6D078",
    "B8A44FE820CC",
    "E8272502493D",
    "E82725029B39",
]

for sn in serial_numbers:
    table.put_item(
        Item={
            "device_serial_number": sn,
            "created_at": datetime.now(timezone.utc).isoformat()
        }
    )
    print(f"Inserted {sn}")

In [ ]:
import boto3

dynamodb = boto3.resource("dynamodb", region_name="eu-west-1")
TABLE_NAME = "device_whitelist"
table = dynamodb.Table(TABLE_NAME)


# Remove a device from the whitelist, for testing
def delete_sn(serial_number : str):
    response = table.delete_item(
        Key={
            "device_serial_number": serial_number
        }
    )   
    print("Delete response:", response)

delete_sn("B8A44FA0DC7A")
